# 2.1.2.	Synthetic - Data Gridding



## Gridding the magnetic data

Magnetic data should be gridded prior to applying transforms and filters. This means that the flight line data must be interpolated on a regular 2D grid. There are numerous possible ways to grid data (Briggs, 1974; Parker et al., 1983). The minimum curvature method is shown to be robust and yield a sensible and smooth result, so we have chosen minimum curvature as the default option to complete the gridding for the synthetic model data. We have provided options below to explore the gridding resolution and the method using the drop down menus.  

The resulting gridded data shows the larger and smaller magnetic bodies, and the magnetic low related to the northwest trending structure cross-cutting the larger magnetic domain.


In [ ]:
## Load all libraries and setup the synthetic problem
from GeoToolkit.Mag import Mag, ProblemSetter, MathUtils, Simulator, DataIO
import numpy as np

dataGrid = DataIO.fetchData(file="./assets/TKC/Synthetic_Forward_TMI.dat", dtype='CSV', checkDir=True, loadFile=True)
display(dataGrid)


In [ ]:
grid = Simulator.dataGriddingWidget(
    dataGrid.result[0], saveAs="./Output/DataGrid",
    Method="minimumCurvature",
    shapeFile='./assets/Synthetic/Synthetic_Zcontours.shp')
display(grid)